# <font color='orange' size='7px'> ***Need Handling***</font>

- sorting the attributes according to a specific attribute.
- categorical attributes generalization techinque.

# <font color='orange' size='7px'>  ***Imports*** </font>

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# <font color='orange' size='7px'>  ***Preprocessing*** </font>

In [3]:
def formalize_data(df):
    '''
    parameter: dataframe: df
    functionality: converting all categorical attributes using label encoder to be ready to algorithm.
    '''
    le = LabelEncoder()
    for column in df.columns:
        if df[column].dtype != 'object': continue
        df[column] = le.fit_transform(df[column])
    return df

# <font color='orange' size='7px'>  ***Algorithm*** </font>

## <font color='orange' size='7px'>  ***Measures*** </font>

In [4]:
def gini_index(y):
    classes = np.unique(y)
    n = len(y)
    gini = 0
    for cls in classes:
        p = np.sum(y == cls) / n
        gini += p * p
    gini =  1 - gini
    return gini

# ***Basic Fundamental unit: Decision stump***

In [5]:
def find_best_split(X, y):
    best_gini = float('inf')
    best_feature = None
    best_threshold = None
    Dy = None
    Dn = None
    Yy = None
    Yn = None

    for feature in range(X.shape[1]):
        # print(f'x: {X}')
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            left_indices = np.where(X[:, feature] <= threshold)[0]
            right_indices = np.where(X[:, feature] > threshold)[0]

            left_gini = gini_index(y[left_indices])
            right_gini = gini_index(y[right_indices])

            weighted_gini = (len(left_indices) / len(y)) * left_gini + (len(right_indices) / len(y)) * right_gini
            print(f'feature: {feature}, threshold: {threshold}, weighted_gini: {weighted_gini}')

            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature
                best_threshold = threshold
                Dy = X[left_indices]
                Dn = X[right_indices]
                Yy = y[left_indices]
                Yn = y[right_indices]

    return best_gini, best_feature, best_threshold, Dy, Dn, Yy, Yn

In [6]:
def decision_stump_fit(X, y):
    best_gini, best_feature, best_threshold, Dy, Dn, Yy, Yn = find_best_split(X, y)
    return best_gini, best_feature, best_threshold, Dy, Dn, Yy, Yn

## ***Decision Tree***

In [7]:
def decision_tree_fit(X, y, max_depth):
    # Base case: if max_depth reached or data is pure
    if y.shape[0] == 0 or len(np.unique(y)) == 1:
        return None, None, None, X, None, y, None

    # Find the best split
    gini, feature, threshold, Dy, Dn, Yy, Yn = find_best_split(X, y)
    if gini != None:
      print(f'best_gini: {gini}\n', f'best_feature: a{feature + 1}\n', f'best_threshold: {threshold}\n', f'depth: {max_depth}\n', Dy, Dn, Yy, Yn)
      print("--------------------------------")

    # Base case: if no split improves purity
    if gini == float('inf'):
        return None, None, None, X, None, y, None

    # Recursive call for left and right branches
    left_branch = decision_tree_fit(Dy, Yy, max_depth + 1)
    right_branch = decision_tree_fit(Dn, Yn, max_depth + 1)

# <font color='orange' size='7px'> ***Testing*** </font>

In [8]:
data = {
    'a1': ['T', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'F'],
    'a2': ['T', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'F'],
    'a3': [1.0, 3.0, 4.0, 5.0, 5.0, 6.0, 7.0, 7.0, 8.0],
}
classes = ['+', '-', '+', '-', '-', '+', '-', '+', '-']

df = pd.DataFrame(data)
df = formalize_data(df)
X = np.array(df)
y = np.array(classes)
decision_tree_fit(X, y, 0)

feature: 0, threshold: 0.0, weighted_gini: 0.34444444444444433
feature: 0, threshold: 1.0, weighted_gini: 0.49382716049382713
feature: 1, threshold: 0.0, weighted_gini: 0.4888888888888889
feature: 1, threshold: 1.0, weighted_gini: 0.49382716049382713
feature: 2, threshold: 1.0, weighted_gini: 0.41666666666666663
feature: 2, threshold: 3.0, weighted_gini: 0.49206349206349215
feature: 2, threshold: 4.0, weighted_gini: 0.4444444444444444
feature: 2, threshold: 5.0, weighted_gini: 0.4888888888888889
feature: 2, threshold: 6.0, weighted_gini: 0.48148148148148145
feature: 2, threshold: 7.0, weighted_gini: 0.4444444444444444
feature: 2, threshold: 8.0, weighted_gini: 0.49382716049382713
best_gini: 0.34444444444444433
 best_feature: a1
 best_threshold: 0.0
 depth: 0
 [[0. 1. 3.]
 [0. 0. 4.]
 [0. 1. 5.]
 [0. 1. 7.]
 [0. 0. 8.]] [[1. 1. 1.]
 [1. 0. 5.]
 [1. 1. 6.]
 [1. 0. 7.]] ['-' '+' '-' '-' '-'] ['+' '-' '+' '+']
--------------------------------
feature: 0, threshold: 0.0, weighted_gini: 0.31